# Lab | Making predictions with logistic regression

In this lab, you will be using the Sakila database of movie rentals.

In order to optimize our inventory, we would like to know which films will be rented. We are asked to create a model to predict it. So we use the information we have from May 2005 to create the model.

Instructions:
1. Create a query or queries to extract the information you think may be relevant for building the prediction model. It should include some film features and some rental features (X).
2. Create a query to get the list of all unique film titles and a boolean indicating if it was rented (rental_date) in May 2005. (Create new column called - 'rented_in_may'). This will be our TARGET (y) variable.
3. Read the data into a Pandas dataframe. At this point you should have 1000 rows. Number of columns depends on the number of features you chose.
4. Analyze extracted features (X) and transform them. You may need to encode some categorical variables, or scale numerical variables.
5. Create a logistic regression model to predict 'rented_in_may' from the cleaned data.
6. Evaluate the results.

In [2]:
# import modules and get pwd
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import getpass  # To get the password without showing the input
password = getpass.getpass()

········


In [3]:
# connecting to sql and get the data
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

# I do it all in one query, where I have 1000 rows
query = '''SELECT DISTINCT f.title, f.release_year, f.rental_duration, f.rental_rate, f.length, f.rating,
max(CASE WHEN r.rental_date BETWEEN '2005-05-01' AND '2005-05-31' THEN 1 ELSE 0 END) AS rented_in_may
FROM film f
LEFT JOIN inventory i ON f.film_id = i.film_id
LEFT JOIN rental r ON i.inventory_id = r.inventory_id
GROUP BY f.film_id'''
movies = pd.read_sql_query(query, engine)
movies.head()

,title,release_year,rental_duration,rental_rate,length,rating,rented_in_may
0,ACADEMY DINOSAUR,2006,6,0.99,86,PG,1
1,ACE GOLDFINGER,2006,3,4.99,48,G,0
2,ADAPTATION HOLES,2006,7,2.99,50,NC-17,0
3,AFFAIR PREJUDICE,2006,5,2.99,117,G,1
4,AFRICAN EGG,2006,6,2.99,130,G,1


In [4]:
# check if 'release_year' is always 2006.
movies['release_year'].value_counts()

release_year
2006    1000
Name: count, dtype: int64

In [5]:
# I wil remove 'release_year' as does not add relevant information to the model.
movies = movies.drop(['release_year'], axis=1)
movies.columns

Index(['title', 'rental_duration', 'rental_rate', 'length', 'rating',
       'rented_in_may'],
      dtype='object')

In [6]:
# Analyze extracted features (X) and transform them. You may need to encode some categorical
# variables, or scale numerical variables.

In [7]:
# Need to X-y split and train-test-split BEFORE I apply transformations, 
# then train transformation on training set only
y = movies['rented_in_may']
X = movies.drop('rented_in_may', axis=1)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [8]:
# I encode the numerical features
from sklearn.preprocessing import MinMaxScaler

X_train_num = X_train.select_dtypes(include = np.number)

# Scaling data
transformer = MinMaxScaler().fit(X_train_num) # need to keep transformer
X_train_normalized = transformer.transform(X_train_num)
X_train_norm = pd.DataFrame(X_train_normalized, columns=X_train_num.columns)
X_train_norm

,rental_duration,rental_rate,length
0,1.00,0.5,0.489209
1,0.25,0.0,0.676259
2,1.00,1.0,0.194245
3,0.50,1.0,0.733813
4,0.25,1.0,0.906475
...,...,...,...
795,1.00,1.0,0.690647
796,0.25,0.5,0.568345
797,1.00,0.0,0.741007
798,0.00,0.5,0.208633


In [9]:
# I encode the categorical features
from sklearn.preprocessing import OneHotEncoder

X_train_cat = X_train.select_dtypes(include = object)

display(X_train_cat['title'].value_counts())
display(X_train_cat['rating'].value_counts())

# I have to drop 'title' from 'X_train_cat' as it won't give the model any relevant information
X_train_cat = X_train_cat.drop(['title'], axis=1)
X_train_cat

encoder = OneHotEncoder(drop='first').fit(X_train_cat)  # This is useful for put our data into the model. The less variables the better so we drop a redundant one.

cols = encoder.get_feature_names_out(input_features=X_train_cat.columns)
onehot_encoded = pd.DataFrame(encoder.transform(X_train_cat).toarray(),columns=cols)

onehot_encoded.head(20)

title
MARRIED GO           1
LOVE SUICIDES        1
LUCK OPUS            1
CONTACT ANONYMOUS    1
DALMATIONS SWEDEN    1
                    ..
PRINCESS GIANT       1
PICKUP DRIVING       1
TRAFFIC HOBBIT       1
EAGLES PANKY         1
HOOSIERS BIRDCAGE    1
Name: count, Length: 800, dtype: int64

rating
PG-13    186
NC-17    174
PG       154
R        148
G        138
Name: count, dtype: int64

,rating_NC-17,rating_PG,rating_PG-13,rating_R
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0
2,0.0,1.0,0.0,0.0
3,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0
5,0.0,1.0,0.0,0.0
6,0.0,1.0,0.0,0.0
7,0.0,0.0,1.0,0.0
8,0.0,0.0,0.0,1.0
9,1.0,0.0,0.0,0.0


In [10]:
# Create a logistic regression model to predict 'rented_in_may' from the cleaned data.

In [11]:
# build X_train and y_train
X_train_transformed = np.concatenate([X_train_norm, onehot_encoded], axis=1)

In [12]:
X_train_transformed

array([[1.        , 0.5       , 0.48920863, ..., 0.        , 0.        ,
        0.        ],
       [0.25      , 0.        , 0.67625899, ..., 0.        , 0.        ,
        1.        ],
       [1.        , 1.        , 0.1942446 , ..., 1.        , 0.        ,
        0.        ],
       ...,
       [1.        , 0.        , 0.74100719, ..., 0.        , 0.        ,
        1.        ],
       [0.        , 0.5       , 0.20863309, ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.5       , 0.9352518 , ..., 0.        , 0.        ,
        0.        ]])

In [13]:
# fit out model
from sklearn.linear_model import LogisticRegression
classification = LogisticRegression(solver='lbfgs').fit(X_train_transformed, y_train)

In [14]:
# We can not make predictions on the X_test yet, we need to perform transformations on the X_test as well

In [15]:
# I encode the numerical features
X_test_num = X_test.select_dtypes(include = np.number)

# Scaling data
transformer = MinMaxScaler().fit(X_test_num) # need to keep transformer
X_test_normalized = transformer.transform(X_test_num)
X_test_norm = pd.DataFrame(X_test_normalized, columns=X_test_num.columns)
X_test_norm

,rental_duration,rental_rate,length
0,1.00,0.5,0.304348
1,0.50,0.5,0.304348
2,0.75,1.0,0.963768
3,0.75,1.0,0.086957
4,0.00,0.0,0.572464
...,...,...,...
195,0.50,1.0,0.318841
196,0.50,1.0,0.833333
197,0.75,0.0,0.913043
198,0.50,0.5,0.942029


In [16]:
# I encode the categorical features
X_test_cat = X_test.select_dtypes(include = object)

# I have to drop 'title' from 'X_train_cat' as it won't give the model any relevant information
X_test_cat = X_test_cat.drop(['title'], axis=1)
X_test_cat

encoder = OneHotEncoder(drop='first').fit(X_test_cat)  # This is useful for put our data into the model. The less variables the better so we drop a redundant one.

cols = encoder.get_feature_names_out(input_features=X_test_cat.columns)
onehot_encoded = pd.DataFrame(encoder.transform(X_test_cat).toarray(),columns=cols)

onehot_encoded.head(20)

,rating_NC-17,rating_PG,rating_PG-13,rating_R
0,0.0,0.0,1.0,0.0
1,1.0,0.0,0.0,0.0
2,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0
5,0.0,0.0,1.0,0.0
6,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,1.0
8,0.0,0.0,1.0,0.0
9,0.0,0.0,1.0,0.0


In [17]:
# we concatenate both X_test_norm and onehot_encoded
X_test_transformed = np.concatenate([X_test_norm, onehot_encoded], axis=1)

In [18]:
X_test_transformed

array([[1.        , 0.5       , 0.30434783, ..., 0.        , 1.        ,
        0.        ],
       [0.5       , 0.5       , 0.30434783, ..., 0.        , 0.        ,
        0.        ],
       [0.75      , 1.        , 0.96376812, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.75      , 0.        , 0.91304348, ..., 0.        , 0.        ,
        0.        ],
       [0.5       , 0.5       , 0.94202899, ..., 1.        , 0.        ,
        0.        ],
       [1.        , 0.5       , 0.23188406, ..., 0.        , 0.        ,
        0.        ]])

In [19]:
# Now we can make predictions on the test set:
predictions = classification.predict(X_test_transformed)
predictions
classification.score(X_test_transformed, y_test)

0.63

In [20]:
# The model predicts 60% of the results well. So it is an usable model, even though I can improve it adding more 
# relevant features or trying other forms of encoding as StandardScaler

In [21]:
# Let's check the confusion matrix
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, predictions)

array([[  0,  74],
       [  0, 126]], dtype=int64)

In [22]:
# predicted | A | B | 
# --------------------------
# actual  A | + |  |  
# --------------------------
#         B |   | + | 
# --------------------------




In [23]:
# I will try the KNN classifier, that looks at nearest neighbours and use the majority to determine class
# This helps to avoid over predicting

# So let's make another model with KNN classifier
from sklearn import neighbors
clf = neighbors.KNeighborsClassifier(n_neighbors=3, weights='uniform')
clf.fit(X_train_transformed, y_train)


KNeighborsClassifier(n_neighbors=3)

In [24]:
# Let's check the predictions
predictions_clf = clf.predict(X_test_transformed)
clf.score(X_test_transformed, y_test)

0.59

In [25]:
confusion_matrix(y_test, predictions_clf)

array([[27, 47],
       [35, 91]], dtype=int64)

In [26]:
# predicted | A | B | C | D |
# --------------------------
# actual  A | + |  |   |   |
# --------------------------
#         B |   | + |   |   |
# --------------------------
#         C |   |   | + |   |
# --------------------------
#         D |   |   |   | + |

In [27]:
# Now I get a similar score (59%), but I am not overpredicting.